In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\MAFC4709\Python_work\RR_model
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001EFD789DC00>, origin='C:\\Users\\MAFC4709\\Python_work\\RR_model\\src\\__init__.py', submodule_search_locations=['C:\\Users\\MAFC4709\\Python_work\\RR_model\\src'])


In [3]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
sql_path = r"C:/Users/MAFC4709/Python_work/RR_model/sql/uns2.sql"

df = load_data(sql_path)
#df = load_data()
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
🔗 Loading data from Oracle...
=== SQL DEBUG ===
File: C:\Users\MAFC4709\Python_work\RR_model\sql\uns2.sql
First 200 chars:
 select * from TBL_RR_UNSECURED2
Params: {}


C:\Users\MAFC4709\Python_work\RR_model\src\db.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np

from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.forecast_plan import forecast_sale_plan_by_mob

from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    add_del_metrics,
    tag_forecast_rows_amount,
)

from src.rollrate.calibration import (
    compute_k_per_product_anchor,
    apply_k_to_forecast,
    apply_k_to_sale_plan,
)

from src.rollrate.seasonality import (
    build_seasonality,
    apply_seasonality_to_lifecycle,
    apply_seasonality_to_sale_plan,
)

from src.config import BUCKETS_CANON, CFG


In [ ]:
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)
print("✔ Transition matrices ready")


In [ ]:
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)
print("✔ Forecast disbursed portfolio done")


In [ ]:
actual_results = get_actual_all_vintages_amount(df)

lifecycle = combine_all_lifecycle_amount(actual_results, forecast_results)

df_lifecycle = lifecycle_to_long_df_amount(lifecycle)
print(df_lifecycle.head())


In [ ]:
df_lifecycle = tag_forecast_rows_amount(df_lifecycle, df)
print("✔ Tagged forecast rows")


In [ ]:
df_lifecycle = add_del_metrics(df_lifecycle, df)
print(df_lifecycle.head())


In [ ]:
sale_plan_df = pd.read_excel("sale_plan.xlsx")
sale_plan_df["VINTAGE_DATE"] = pd.to_datetime(sale_plan_df["VINTAGE_DATE"])

# Chuẩn hóa SCORE
def normalize_score(x):
    if pd.isna(x):
        return "ALL"
    s = str(x).strip().upper()
    return "ALL" if s in ["", "NA", "NAN", "NULL"] else s

sale_plan_df["RISK_SCORE"] = sale_plan_df["RISK_SCORE"].apply(normalize_score)

sale_plan_df.head()


In [ ]:
states = list(dict.fromkeys(list(BUCKETS_CANON) + ["WRITEOFF", "PREPAY"]))

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=states,
)

print("✔ Forecast SALE PLAN done")
sale_plan_fc.head()


In [ ]:
seasonality = build_seasonality(df_lifecycle)
print("✔ Seasonality ready")


In [ ]:
df_lifecycle = apply_seasonality_to_lifecycle(df_lifecycle, seasonality)
sale_plan_fc = apply_seasonality_to_sale_plan(sale_plan_fc, seasonality)

print("✔ Applied seasonality")


In [ ]:
# Gán mapping anchor MOB
anchor_map = {
    "CDLPIL": 12,
    "TWLPIL": 12,
    "SPLPIL": 12,
    # default cho các product khác
}

k_dict = compute_k_per_product_anchor(
    df_raw=df,
    forecast_results=forecast_results,
    anchor_map=anchor_map,
)

print(k_dict)


In [ ]:
df_lifecycle = apply_k_to_forecast(df_lifecycle, k_dict)
sale_plan_fc = apply_k_to_sale_plan(sale_plan_fc, k_dict)


In [ ]:
from src.rollrate.export import export_lifecycle_all_products_one_file

# Xây actual_info cho export
actual_info = df_lifecycle.groupby(["PRODUCT_TYPE","VINTAGE_DATE"])["ACTUAL_MAX_MOB"].max().to_dict()

export_lifecycle_all_products_one_file(
    df_del_prod=df_lifecycle,
    actual_info=actual_info,
    filename="Lifecycle_Report.xlsx"
)
